In [1]:
import os
import json
from datetime import datetime
from tempfile import TemporaryDirectory
from websockets.sync.client import connect as ws_connect
from contextlib import asynccontextmanager
from pathlib import Path
from fastapi import FastAPI
from fastapi.responses import HTMLResponse
#import uvicorn
import asyncio
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from docx import Document as DocxDocument
from PyPDF2 import PdfReader
from dotenv import load_dotenv

In [1]:
import autogen
from autogen import AssistantAgent, UserProxyAgent
from autogen.io.websockets import IOWebsockets


In [2]:
from prompt import data_collection_prompt, estimation_prompt

In [3]:
# LLM Configuration
llm_config = {
    "timeout": 600,
    "config_list": autogen.config_list_from_json("OAI_CONFIG_LIST"),
    "temperature": 0,
}

In [4]:
 data_collection_agent = AssistantAgent(
        name="data_collection_agent",
        system_message=data_collection_prompt,
        llm_config=llm_config,
        # human_input_mode="ALWAYS",
        is_termination_msg=lambda msg: "terminate" in msg["content"].lower() or "exit" in msg["content"].lower(),
    )

In [5]:
estimation_agent = AssistantAgent(
        name="estimation_agent",
        system_message=estimation_prompt,
        llm_config=llm_config,
        human_input_mode="NEVER",
    )

In [7]:
user_proxy = UserProxyAgent(
        name="user_proxy",
        human_input_mode="ALWAYS",
        llm_config=llm_config,
        code_execution_config=False,
        system_message="You are a helpful assistant.",
        is_termination_msg=lambda msg: "terminate" in msg["content"].lower()or "exit" in msg["content"].lower(),
    )

In [8]:
chat_results = user_proxy.initiate_chats(
        [
            {
                "recipient": data_collection_agent,
                "message": "You are a Technical architect. Your job is to arrive at the tech stack for a problem statement and also arrive at estimates for the application in scope.",
                "clear_history": True,
                "silent": False,
                "summary_method": "last_msg",
            },
            {
                "recipient": estimation_agent,
                "message": "Please generate estimates based on the collected information.",
                "summary_method": "reflection_with_llm",
            },
        ]
    )


********************************************************************************
Starting a new chat....

********************************************************************************
user_proxy (to data_collection_agent):

You are a Technical architect. Your job is to arrive at the tech stack for a problem statement and also arrive at estimates for the application in scope.

--------------------------------------------------------------------------------
data_collection_agent (to user_proxy):

To begin gathering the necessary information for your project, could you please describe the application's purpose and its main features? Additionally, who is the target audience for this application? Are there any specific requirements, timelines, or budget limits that we should be aware of?

--------------------------------------------------------------------------------
user_proxy (to data_collection_agent):

I want to extract content from HIMSS website, all the news articles should be ex